In [4]:
from platform import python_version

print(python_version())

3.9.7


# Data segmentation

# Feature extraction

In [5]:
import numpy as np
import scipy
from scipy import stats
from scipy.integrate import trapz
from scipy.fft import fft

global num_features
num_features = 10
def feature_extraction(data):
        freq = 100
        mean_array = np.zeros((len(data),2), dtype = float)
        median_array = np.zeros((len(data),2), dtype = float)
        std_array = np.zeros((len(data),2), dtype = float)
        min_val_array = np.zeros((len(data),2), dtype = float)
        max_val_array = np.zeros((len(data),2), dtype = float)
        sum_val_array = np.zeros((len(data),2), dtype = float)
        kurtosis_array = np.zeros((len(data),2), dtype = float)
        skewness_array = np.zeros((len(data),2), dtype = float)
        for i in range(len(data)):
                mean_array[i] = np.mean(data[i], axis=0)
                median_array[i] = np.median(data[i], axis=0)
                std_array[i] = np.std(data[i], axis=0)
                min_val_array[i] = np.min(data[i], axis=0)
                max_val_array[i] = np.max(data[i], axis=0)
                sum_val_array[i] = np.sum(data[i], axis=0)/freq
                
                kurtosis_array[i] = scipy.stats.kurtosis(data[i], axis=0)       
                skewness_array[i] = scipy.stats.skew(data[i], axis=0)


        fft_sums = np.zeros((len(data),2), dtype = float)

        fft_freqs = np.zeros((len(data),2), dtype = float) ###Czy tu aby na pewno powinien być dloat, a nie liczba urojona

        #Flattening the array to shape (a*b, c), where c is the number of sensors
        flat_arr = data.reshape(len(data)*len(data[1]), len(data[0][1]))

        
        k = 0
        for k in range(len(data)):
                
                chunk = flat_arr[k * len(data[1]) : (k + 1) * len(data[1]), ]
                
                sensor_1 = chunk[:,0]
                sensor_2 = chunk[:,1]
                
                
                sp1 = np.fft.fft(sensor_1)
                ps1 = np.abs(sp1)**2
                sp2 = np.fft.fft(sensor_2)
                ps2 = np.abs(sp2)**2
                
                
                # Define the frequency range of interest
                sensor_1_freq = np.fft.fftfreq(len(chunk), 1/freq)
                
                idx1 = np.logical_and(sensor_1_freq >= 0, sensor_1_freq <= freq)

                # Integrate the power spectrum over the frequency range of interest
                area1 = trapz(ps1[idx1], sensor_1_freq[idx1])


                # Define the frequency range of interest
                sensor_2_freq = np.fft.fftfreq(len(chunk), 1/freq)
                
                idx2 = np.logical_and(sensor_2_freq >= 0, sensor_2_freq <= freq)

                # Integrate the power spectrum over the frequency range of interest
                area2 = trapz(ps2[idx1], sensor_2_freq[idx1])


                fft_sums[k] = [area1, area2]
                
                
                argmax_ind_1 = np.arange(len(sp1))
                argmax_list_1 = argmax_ind_1[np.argsort(-np.abs(sp1))]
                
                max_power_frequency_1 = argmax_list_1[0] * (freq / len(chunk))
                
                
                argmax_ind_2 = np.arange(len(sp2))
                argmax_list_2 = argmax_ind_2[np.argsort(-np.abs(sp2))]
                
                
                array_len_1 = len(argmax_list_1)
                
                for ind_1 in range(array_len_1):
                        
                        if (argmax_list_1[ind_1] * (freq / len(chunk)) <= 1):
                                
                                continue
                        
                        elif (argmax_list_1[ind_1] * (freq / len(chunk)) > 1):
                                max_power_frequency_1 = argmax_list_1[ind_1] * (freq / len(chunk))

                                break
                        
                array_len_2 = len(argmax_list_2)

                for ind_2 in range(array_len_2):
                        
                        if (argmax_list_2[ind_2] * (freq / len(chunk)) <= 1):
                                
                                continue
                        
                        elif (argmax_list_2[ind_2] * (freq / len(chunk)) > 1):
                                max_power_frequency_2 = argmax_list_2[ind_2] * (freq / len(chunk))
        
                                break
           
        
        
                fft_freqs[k] = [max_power_frequency_1, max_power_frequency_2]


        global feature_names
        feature_names = ['mean', 'mean_2', 'median', 'median_2', 'std', 'std_2', 'min_val', 'min_val_2',\
                'max_val', 'max_val_2', 'sum_val', 'sum_val_2', 'kurtosis', 'kurtosis_2', 'skew', 'skew_2', 'fft_sum', 'fft_sum_2', 'argmax_freq', 'argmax_freq_2']
        
        features = np.zeros((len(data), num_features))        

        features = np.concatenate((mean_array, median_array, std_array, min_val_array, max_val_array, sum_val_array, kurtosis_array, skewness_array, fft_sums, fft_freqs), axis = 1)
        
        return features    

In [18]:
import os
data_dir = './CreatedFiles/Segmentation/'


for filename in os.listdir(data_dir):
    if filename.endswith('_B.npy'):
        file_loaded = np.load(data_dir + filename)
        extracted = feature_extraction(file_loaded)
        
        location_FE_B = f'./CreatedFiles/Extracted_Features/{filename}'
        np.save(location_FE_B, extracted)
    elif filename.endswith('_R.npy'):
        file_loaded = np.load(data_dir + filename)
        extracted = feature_extraction(file_loaded)
        
        location_FE_R = f'./CreatedFiles/Extracted_Features/{filename}'
        np.save(location_FE_R, extracted)
        continue
    else:
        print('Error file not found')
        break


In [ ]:
def labeling():
    extracted_features = []
    for i in range(len(position)):
        extracted_features.append(feature_extraction(position[i]))
        #print(extracted_features)
    
    position_0 = feature_extraction(position[0])
    position_1 = feature_extraction(position[1])
    
    labels_0 = np.zeros(position_0.shape[0])
    labels_1 = np.ones(position_1.shape[0])
    
    complete_feature_dataset = np.concatenate((position_0,position_1))
    complete_label_dataset = np.concatenate((labels_0, labels_1))
    
    return complete_feature_dataset, complete_label_dataset

In [1]:
import re
import os
data_dir = './CreatedFiles/Extracted_Features/'


for filename in os.listdir(data_dir):
    if filename.endswith('_B.npy'):
        
        file_loaded = np.load(data_dir + filename)
        labels = np.zeros(file_loaded.shape[0])
        
        label_file = re.sub('(\.npy)','',filename)
        #location_FE_B = f'./CreatedFiles/Labels/{filename}'
        #np.save(location_FE_B, extracted)
        
    elif filename.endswith('_R.npy'):
        
        #file_loaded = np.load(data_dir + filename)
        #labels = np.zeros(file_loaded.shape[0])
    
        #location_FE_R = f'./CreatedFiles/Labels/{filename}'
        #np.save(location_FE_R, extracted)
        continue
    else:
        print('Filename error')
        break

NameError: name 'np' is not defined

#labels = np.squeeze(labels)

#%debug

###Creating an additional table called Saved_data for further actions

In [ ]:
#Saved_data = complete_feature_dataset

In [ ]:
#Saved_data.shape

In [ ]:
#data_to_save = np.array(data_to_save) # this is the 3D array
#data_to_save = data_to_save.reshape(data_to_save.shape[0], -1) # reshape to 2D array

## Data shuffling

In [ ]:
from sklearn.preprocessing import LabelEncoder
def data_shuffling():
    dataset = labeling()
    shuffler = np.random.permutation(len(dataset[0]))
    X = dataset[0][shuffler]
    y = dataset[1][shuffler]

    return X,y
#le = LabelEncoder()
#y = le.fit_transform(y)

In [ ]:
#data_shuffling()

# Recursive feature elimination

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_absolute_error

In [ ]:
def feature_elimination(n = i):
    # Create the Random Forest classifier
    rf = RandomForestClassifier()

    # Perform feature selection using RFE
    rfe = RFE(estimator=rf, n_features_to_select=n, step=1)
    shuffled_dataset = data_shuffling()
    X = shuffled_dataset[0]
    y = shuffled_dataset[1]
    rfe.fit(X, y)

    # Get the selected feature indices
    selected_features = rfe.support_
    selected_features_indices = np.where(selected_features)[0]
    print(selected_features_indices)
    print('Number of features selected: %d' % (n))
    
    names_selected_features = []


    for i in selected_features_indices:
        names_selected_features.append(feature_names[i])
    
    
    print('Features selected: ')
    print(names_selected_features)
    
    le = LabelEncoder()
    y = le.fit_transform(y)
    # Use the selected features to train and evaluate the classifier
    global X_selected
    X_selected = X[:, selected_features]
    global X_train, X_test, y_train, y_test
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

    
    
    # Train the Random Forest classifier
    rf.fit(X_train, y_train)

    # Evaluate the classifier
    y_pred = rf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    mae = mean_absolute_error(y_test, y_pred)

    print("Accuracy: %.4f%%" % (accuracy * 100.0))
    print("Precision: %.4f%%" % (precision * 100.0))
    print("Recall: %.4f%%" % (recall * 100.0))
    print("Mean Absolute Error:", mae, end = '\n')
    
    return X_train, X_test, y_train, y_test, X_selected, y

In [ ]:
#def xyz():
for i in range(num_features*2,0,-1):
    feature_elimination(i)

In [ ]:
#Code opens a mini window in the upper part of the screen
n = int(input("Choose, with how many features do you want to continue"))
feature_elimination(n)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

In [ ]:
print(X_train.size)
print(X_test.size)

In [ ]:
print(y_train.size)
print(y_test.size)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
# create the model
rfc = RandomForestClassifier()

# fit the model to the training data
rfc.fit(X_train, y_train)

# make predictions on the test set
rfc_prediction = rfc.predict(X_test)


acc = accuracy_score(y_test, rfc_prediction)
print("Accuracy: %.2f%%" % (acc * 100.0))
precision = precision_score(y_test, rfc_prediction)
print("Precision: %.2f%%" % (precision * 100.0))
recall = recall_score(y_test, rfc_prediction)
print("Recall: %.2f%%" % (recall * 100.0))

In [ ]:
confusion_matrix = metrics.confusion_matrix(y_test, rfc_prediction)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()

In [ ]:
#!pip install xgboost

In [ ]:
import xgboost as xgb

# create the model
model = xgb.XGBClassifier()

# fit the model to the training data
model.fit(X_train, y_train)

# make predictions on the test set
y_pred = model.predict(X_test)

# evaluate the model's performance
acc = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (acc * 100.0))
precision = precision_score(y_test, y_pred)
print("Precision: %.2f%%" % (precision * 100.0))
recall = recall_score(y_test, y_pred)
print("Recall: %.2f%%" % (recall * 100.0))

In [ ]:
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()

In [ ]:
from sklearn import svm

# create the model
clf = svm.SVC(kernel='rbf', C=1, gamma='scale')

# fit the model to the training data
clf.fit(X_train, y_train)

# make predictions on the test set
y_pred = clf.predict(X_test)

# evaluate the model's performance
acc = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (acc * 100.0))
precision = precision_score(y_test, y_pred)
print("Precision: %.2f%%" % (precision * 100.0))
recall = recall_score(y_test, y_pred)
print("Recall: %.2f%%" % (recall * 100.0))


In [ ]:
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification

# K Fold Cross Validation

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# Define the number of folds
n_folds = 5

# Create an instance of the KFold class
kf = KFold(n_splits=n_folds, random_state=42, shuffle=True)


In [ ]:
# Define the RFC model
rfc = RandomForestClassifier()

# Initialize a list to store the accuracy scores
acc_scores_RFC = []
precision_scores_RFC = []
recall_scores_RFC = []

# Perform the K-fold cross-validation
for train_index, test_index in kf.split(X):
    # Split the data into train and test sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Fit the model to the training data
    rfc.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = rfc.predict(X_test)

    # Calculate the scores
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    acc_scores_RFC.append(acc)
    precision_scores_RFC.append(precision)
    recall_scores_RFC.append(recall)

# Print the mean scores
print("Mean accuracy:", np.mean(acc_scores_RFC))
print("Mean precision:", np.mean(precision_scores_RFC))
print("Mean recall:", np.mean(recall_scores_RFC))

In [ ]:
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()

In [ ]:
# Define the XGboost model
model = xgb.XGBClassifier()

# Initialize a list to store the accuracy scores
acc_scores_XGB = []
precision_scores_XGB = []
recall_scores_XGB = []

# Perform the K-fold cross-validation
for train_index, test_index in kf.split(X):
    # Split the data into train and test sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Fit the model to the training data
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate the scores
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    acc_scores_XGB.append(acc)
    precision_scores_XGB.append(precision)
    recall_scores_XGB.append(recall)

# Print the mean scores
print("Mean accuracy:", np.mean(acc_scores_XGB))
print("Mean precision:", np.mean(precision_scores_XGB))
print("Mean recall:", np.mean(recall_scores_XGB))

In [ ]:
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()

In [ ]:
# Define the SVM model
SVM = svm.SVC(kernel='rbf', C=1, gamma='scale')

# Initialize a list to store the accuracy scores
acc_scores_SVC = []
precision_scores_SVC = []
recall_scores_SVC = []

# Perform the K-fold cross-validation
for train_index, test_index in kf.split(X):
    # Split the data into train and test sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Fit the model to the training data
    SVM.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = SVM.predict(X_test)

    # Calculate the scores
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    acc_scores_SVC.append(acc)
    precision_scores_SVC.append(precision)
    recall_scores_SVC.append(recall)

# Print the mean scores
print("Mean accuracy:", np.mean(acc_scores_SVC))
print("Mean precision:", np.mean(precision_scores_SVC))
print("Mean recall:", np.mean(recall_scores_SVC))

In [ ]:
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()

Power spectrum

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Compute the FFT of the signal along the first axis for channel 0
fft0 = np.fft.fft(Saved_data[:,:,0], axis=0)

# Compute the FFT of the signal along the first axis for channel 1
fft1 = np.fft.fft(Saved_data[:,:,1], axis=0)

# Compute the power spectrum for channel 0
power_spectrum0 = np.abs(fft0)**2

# Compute the power spectrum for channel 1
power_spectrum1 = np.abs(fft1)**2

plots = 25
# Plot the power spectrum of 25 slices along the first axis for channel 0
for i in range(plots):
    plt.plot(power_spectrum0[:, i])
    plt.xlabel('Frequency (bins)')
    plt.ylabel('Power')
    plt.title(f'Power Spectrum of Signal Channel 0 (Slice {i})')
    plt.show()

# Plot the power spectrum of 5 slices along the first axis for channel 1
for i in range(plots):
    plt.plot(power_spectrum1[:, i])
    plt.xlabel('Frequency (bins)')
    plt.ylabel('Power')
    plt.title(f'Power Spectrum of Signal Channel 1 (Slice {i})')
    plt.show()

Confusion matrix

In [ ]:
# Compute the FFT of the signal along the first axis for channel 0
fft0 = np.fft.fft(Saved_data[:,:,0], axis=0)

# Compute the FFT of the signal along the first axis for channel 1
fft1 = np.fft.fft(Saved_data[:,:,1], axis=0)

# Compute the power spectrum for channel 0
power_spectrum0 = np.abs(fft0)**2

# Compute the power spectrum for channel 1
power_spectrum1 = np.abs(fft1)**2

# Sampling rate
sampling_rate = 200

# Number of samples
N = Saved_data.shape[0]

# Compute the frequencies in Hz for the power spectrum
frequencies0 = np.fft.fftfreq(N, 1/sampling_rate)
frequencies1 = np.fft.fftfreq(N, 1/sampling_rate)

plots = 25
# Plot the power spectrum of 25 slices along the first axis for channel 0
for i in range(plots):
    plt.plot(frequencies0, power_spectrum0[:, i])
    plt.xlabel('Frequency (Hz)')
    plt.xlim(left=0)
    plt.ylabel('Power')
    plt.title(f'Power Spectrum of Signal Channel 0 (Slice {i})')
    plt.show()

# Plot the power spectrum of 5 slices along the first axis for channel 1
for i in range(plots):
    plt.plot(frequencies1, power_spectrum1[:, i])
    plt.xlabel('Frequency (Hz)')
    plt.xlim(left=0)
    plt.ylabel('Power')
    plt.title(f'Power Spectrum of Signal Channel 1 (Slice {i})')
    plt.show()

Plots prepared for newly preprocessed data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Compute the FFT of the signal along the first axis for each sensor
fft = np.fft.fft(new_array, axis=0)

# Compute the power spectrum for each sensor
power_spectrum = np.abs(fft)**2

# Sampling rate
sampling_rate = 1

# Number of samples
N = new_array.shape[0]

# Compute the frequencies in Hz for the power spectrum
frequencies = np.fft.fftfreq(N, 1/sampling_rate)

# Plot the power spectrum for each sensor
for i in range(new_array.shape[1]):
    plt.plot(frequencies, power_spectrum[:, i])
    plt.xlabel('Frequency (Hz)')
    plt.xlim(left=0)
    plt.ylabel('Power')
    plt.title(f'Power Spectrum of Sensor {i}')
    plt.show()